In [5]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('inventory_management.db')
cursor = conn.cursor()


In [6]:
# Create Products Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        product_id INTEGER PRIMARY KEY AUTOINCREMENT,
        product_name TEXT,
        category TEXT,
        current_stock INTEGER,
        reorder_level INTEGER,
        reorder_quantity INTEGER
    )
''')





In [7]:
# Insert initial product data
products_data = [
    ('Laptop', 'Electronics', 15, 10, 20),
    ('Smartphone', 'Electronics', 30, 15, 40),
    ('Headphones', 'Accessories', 100, 50, 100),
    ('Chair', 'Furniture', 8, 5, 10),
    ('Desk', 'Furniture', 5, 3, 5)
]

In [8]:
cursor.executemany('INSERT INTO products (product_name, category, current_stock, reorder_level, reorder_quantity) VALUES (?, ?, ?, ?, ?)', products_data)

# Commit the changes
conn.commit()
print("Products table created and populated with initial data.")

Products table created and populated with initial data.


In [9]:
# Function to display inventory status
def show_inventory():
    cursor.execute('SELECT * FROM products')
    rows = cursor.fetchall()
    print("Current Inventory:")
    for row in rows:
        print(f"Product ID: {row[0]}, Name: {row[1]}, Stock: {row[3]}, Reorder Level: {row[4]}")

In [10]:
# Function to process sales
def process_sale(product_id, quantity_sold):
    cursor.execute('SELECT current_stock FROM products WHERE product_id = ?', (product_id,))
    current_stock = cursor.fetchone()[0]
    
    if current_stock >= quantity_sold:
        new_stock = current_stock - quantity_sold
        cursor.execute('UPDATE products SET current_stock = ? WHERE product_id = ?', (new_stock, product_id))
        conn.commit()
        print(f"Sale processed! New stock for Product ID {product_id} is {new_stock}.")
    else:
        print(f"Insufficient stock for Product ID {product_id}. Only {current_stock} items available.")

In [11]:
# Function to generate low-stock alerts
def check_low_stock():
    cursor.execute('SELECT * FROM products WHERE current_stock <= reorder_level')
    rows = cursor.fetchall()
    
    if rows:
        print("\nLow Stock Alert:")
        for row in rows:
            print(f"Product ID: {row[0]}, Name: {row[1]}, Current Stock: {row[3]}, Reorder Level: {row[4]}")
    else:
        print("All products have sufficient stock.")

In [12]:
# Function to restock products
def restock_product(product_id):
    cursor.execute('SELECT current_stock, reorder_quantity FROM products WHERE product_id = ?', (product_id,))
    result = cursor.fetchone()
    current_stock, reorder_quantity = result
    new_stock = current_stock + reorder_quantity
    
    cursor.execute('UPDATE products SET current_stock = ? WHERE product_id = ?', (new_stock, product_id))
    conn.commit()
    
    print(f"Product ID {product_id} restocked! New stock is {new_stock}.")

In [13]:
# Function to automatically restock low-stock items
def auto_restock():
    cursor.execute('SELECT product_id, current_stock, reorder_quantity FROM products WHERE current_stock <= reorder_level')
    rows = cursor.fetchall()
    
    if rows:
        print("\nAuto Restocking:")
        for row in rows:
            product_id, current_stock, reorder_quantity = row
            new_stock = current_stock + reorder_quantity
            cursor.execute('UPDATE products SET current_stock = ? WHERE product_id = ?', (new_stock, product_id))
            print(f"Product ID {product_id} restocked to {new_stock} items.")
        conn.commit()
    else:
        print("No products need restocking.")

In [14]:
# Function to add new products to inventory
def add_product(product_name, category, current_stock, reorder_level, reorder_quantity):
    cursor.execute('INSERT INTO products (product_name, category, current_stock, reorder_level, reorder_quantity) VALUES (?, ?, ?, ?, ?)',
                   (product_name, category, current_stock, reorder_level, reorder_quantity))
    conn.commit()
    print(f"Product '{product_name}' added to inventory.")

In [15]:
# Display initial inventory
show_inventory()


Current Inventory:
Product ID: 1, Name: Laptop, Stock: 15, Reorder Level: 10
Product ID: 2, Name: Smartphone, Stock: 30, Reorder Level: 15
Product ID: 3, Name: Headphones, Stock: 100, Reorder Level: 50
Product ID: 4, Name: Chair, Stock: 8, Reorder Level: 5
Product ID: 5, Name: Desk, Stock: 5, Reorder Level: 3


In [16]:
# Simulate a sale
process_sale(product_id=1, quantity_sold=5) 

Sale processed! New stock for Product ID 1 is 10.


In [17]:
# Check for low-stock alerts
check_low_stock()


Low Stock Alert:
Product ID: 1, Name: Laptop, Current Stock: 10, Reorder Level: 10


In [18]:
# Manually restock a product
restock_product(product_id=1) 

Product ID 1 restocked! New stock is 30.


In [19]:
# Automatically restock products below reorder level
auto_restock()

No products need restocking.


In [20]:
# Add a new product
add_product('Monitor', 'Electronics', 20, 10, 15)

Product 'Monitor' added to inventory.


In [21]:
# Display updated inventory
show_inventory()

Current Inventory:
Product ID: 1, Name: Laptop, Stock: 30, Reorder Level: 10
Product ID: 2, Name: Smartphone, Stock: 30, Reorder Level: 15
Product ID: 3, Name: Headphones, Stock: 100, Reorder Level: 50
Product ID: 4, Name: Chair, Stock: 8, Reorder Level: 5
Product ID: 5, Name: Desk, Stock: 5, Reorder Level: 3
Product ID: 6, Name: Monitor, Stock: 20, Reorder Level: 10
